In [1]:
import numpy as np
import gradio as gr
import cv2
from scipy.spatial import distance
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [2]:
model = load_model('masknet.h5')

In [ ]:
try:
  def classify_image(inp):
      mask_label = {0:'MASK',1:'NO MASK'}
      dist_label = {0:(0,255,0),1:(255,0,0)}
      face_model = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
      MIN_DISTANCE = 130
      img = inp
      faces = face_model.detectMultiScale(img,scaleFactor=1.1, minNeighbors=4) #returns a list of (x,y,w,h) tuples

      if len(faces)>0:
          label = [0 for i in range(len(faces))]
          for i in range(len(faces)-1):
              for j in range(i+1, len(faces)):
                  dist = distance.euclidean(faces[i][:2],faces[j][:2])
                  if dist<MIN_DISTANCE:
                      label[i] = 1
                      label[j] = 1
          new_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) #colored output image
          for i in range(len(faces)):
              (x,y,w,h) = faces[i]
              crop = new_img[y:y+h,x:x+w]
              crop = cv2.resize(crop,(128,128))
              crop = np.reshape(crop,[1,128,128,3])/255.0
              mask_result = model.predict(crop)
   
          return f"{mask_label[mask_result.argmax()]} : {round(max(mask_result[0])*100,1)}%"
      else:
          return "Try with Another Picture"
  image = gr.inputs.Image(shape=(224, 224))
  label = gr.outputs.Label(num_top_classes=1)
  info="Abstract:\n The spread of Coronavirus started at the end of the year 2019 from the city of China, Wuhan. The continuous spread of the virus forced governments of various countries to put lockdown for several months. It has been observed that wearing a face mask can actually prevent the transmission of this deadly virus. In the future, we have to use a face mask as a preventive measure for any such viruses. However, manually it is very difficult to keep track of a person wearing a mask or not. And here technology plays a very crucial role. This project highlights the importance of deep learning especially object detection. We introduce a deep learning model that will detect faces and can predict whether the person is wearing a mask or not."
  gr.Interface(fn=classify_image, inputs=image, outputs=label, capture_session=True,examples=["mo-salah.jpg","cr7.jpg","lana.jpg"],title="Face Mask Detection",article=info,thumbnail="cover.png").launch(debug=True,share=True)
except KeyboardInterrupt:
      pass  # do nothing here

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://7ee49a5e7f033fd7.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


1/1 [==============================] - 0s 105ms/step
